In [1]:
import numpy as np
import os
import sys
import time
import argparse
import json
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import torch
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable

sys.path.insert(0,'../modules')
from sample_generator import *
from data_prov import *
from model import *
from bbreg import *
from options import *
from gen_config import *

In [2]:
np.random.seed(123)
torch.manual_seed(456)
torch.cuda.manual_seed(789)

In [3]:
def forward_samples(model, image, samples, out_layer='conv3'):
    model.eval()
    extractor = RegionExtractor(image, samples, opts['img_size'], opts['padding'], opts['batch_test'])
    for i, regions in enumerate(extractor):
        regions = Variable(regions)
        if opts['use_gpu']:
            regions = regions.cuda()
        feat = model(regions, out_layer=out_layer)
        if i==0:
            feats = feat.data.clone()
        else:
            feats = torch.cat((feats,feat.data.clone()),0)
    return feats

In [4]:
def set_optimizer(model, lr_base, lr_mult=opts['lr_mult'], momentum=opts['momentum'], w_decay=opts['w_decay']):
    params = model.get_learnable_params()
    param_list = []
    for k, p in params.iteritems():
        lr = lr_base
        for l, m in lr_mult.iteritems():
            if k.startswith(l):
                lr = lr_base * m
        param_list.append({'params': [p], 'lr':lr})
    optimizer = optim.SGD(param_list, lr = lr, momentum=momentum, weight_decay=w_decay)
    return optimizer

In [5]:
def train(model, criterion, optimizer, pos_feats, neg_feats, maxiter, in_layer='fc4'):
    model.train()
    
    batch_pos = opts['batch_pos']
    batch_neg = opts['batch_neg']
    batch_test = opts['batch_test']
    batch_neg_cand = max(opts['batch_neg_cand'], batch_neg)

    pos_idx = np.random.permutation(pos_feats.size(0))
    neg_idx = np.random.permutation(neg_feats.size(0))
    while(len(pos_idx) < batch_pos*maxiter):
        pos_idx = np.concatenate([pos_idx, np.random.permutation(pos_feats.size(0))])
    while(len(neg_idx) < batch_neg_cand*maxiter):
        neg_idx = np.concatenate([neg_idx, np.random.permutation(neg_feats.size(0))])
    pos_pointer = 0
    neg_pointer = 0

    for iter in range(maxiter):

        # select pos idx
        pos_next = pos_pointer+batch_pos
        pos_cur_idx = pos_idx[pos_pointer:pos_next]
        pos_cur_idx = pos_feats.new(pos_cur_idx).long()
        pos_pointer = pos_next

        # select neg idx
        neg_next = neg_pointer+batch_neg_cand
        neg_cur_idx = neg_idx[neg_pointer:neg_next]
        neg_cur_idx = neg_feats.new(neg_cur_idx).long()
        neg_pointer = neg_next

        # create batch
        batch_pos_feats = Variable(pos_feats.index_select(0, pos_cur_idx))
        batch_neg_feats = Variable(neg_feats.index_select(0, neg_cur_idx))

        # hard negative mining
        if batch_neg_cand > batch_neg:
            model.eval()
            for start in range(0,batch_neg_cand,batch_test):
                end = min(start+batch_test,batch_neg_cand)
                score = model(batch_neg_feats[start:end], in_layer=in_layer)
                if start==0:
                    neg_cand_score = score.data[:,1].clone()
                else:
                    neg_cand_score = torch.cat((neg_cand_score, score.data[:,1].clone()),0)

            _, top_idx = neg_cand_score.topk(batch_neg)
            batch_neg_feats = batch_neg_feats.index_select(0, Variable(top_idx))
            model.train()
        
        # forward
        pos_score = model(batch_pos_feats, in_layer=in_layer)
        neg_score = model(batch_neg_feats, in_layer=in_layer)
        
        # optimize
        loss = criterion(pos_score, neg_score)
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), opts['grad_clip'])
        optimizer.step()

        #print "Iter %d, Loss %.4f" % (iter, loss.data[0])

In [6]:
def save_fig(image, trbb, gtbb, ind, type = 1):
    draw = ImageDraw.Draw(image)
    draw.line([trbb[0], trbb[1], trbb[0], trbb[1] + trbb[3], trbb[0] + trbb[2], trbb[1] + trbb[3], trbb[0] + trbb[2], trbb[1], trbb[0], trbb[1]], fill = (0,255,0), width = 2)
    if type is 1:
        draw.line([gtbb[0], gtbb[1], gtbb[0], gtbb[1] + gtbb[3], gtbb[0] + gtbb[2], gtbb[1] + gtbb[3], gtbb[0] + gtbb[2], gtbb[1], gtbb[0], gtbb[1]], fill = (255,0,0), width = 2)
    del draw
    image.save(os.path.join(savefig_dir,'%04d.jpg'%(ind)), "JPEG")
    del image

In [7]:
def run_mdnet(img_list, init_bbox, gt=None, savefig_dir='', display=False):
    
    # Init bbox
    target_bbox = np.array(init_bbox)
    result = np.zeros((len(img_list),4))
    result_bb = np.zeros((len(img_list),4))
    result[0] = target_bbox
    result_bb[0] = target_bbox

    # Init model
    model = MDNet(opts['model_path'])
    if opts['use_gpu']:
        model = model.cuda()
    model.set_learnable_params(opts['ft_layers'])
    
    # Init criterion and optimizer 
    criterion = BinaryLoss()
    init_optimizer = set_optimizer(model, opts['lr_init'])
    update_optimizer = set_optimizer(model, opts['lr_update'])

    tic = time.time()
    # Load first image
    image = Image.open(img_list[0]).convert('RGB')
    
    # Train bbox regressor
    bbreg_examples = gen_samples(SampleGenerator('uniform', image.size, 0.3, 1.5, 1.1),
                                 target_bbox, opts['n_bbreg'], opts['overlap_bbreg'], opts['scale_bbreg'])
    bbreg_feats = forward_samples(model, image, bbreg_examples)
    bbreg = BBRegressor(image.size)
    bbreg.train(bbreg_feats, bbreg_examples, target_bbox)
    
    success_fra = [0];

    # Draw pos/neg samples
    pos_examples = gen_samples(SampleGenerator('gaussian', image.size, 0.1, 1.2),
                               target_bbox, opts['n_pos_init'], opts['overlap_pos_init'])

    neg_examples = np.concatenate([
                    gen_samples(SampleGenerator('uniform', image.size, 1, 2, 1.1), 
                                target_bbox, opts['n_neg_init']//2, opts['overlap_neg_init']),
                    gen_samples(SampleGenerator('whole', image.size, 0, 1.2, 1.1),
                                target_bbox, opts['n_neg_init']//2, opts['overlap_neg_init'])])
    neg_examples = np.random.permutation(neg_examples)

    # Extract pos/neg features
    pos_feats = forward_samples(model, image, pos_examples)
    neg_feats = forward_samples(model, image, neg_examples)
    feat_dim = pos_feats.size(-1)

    # Initial training
    train(model, criterion, init_optimizer, pos_feats, neg_feats, opts['maxiter_init'])
    
    # Init sample generators
    sample_generator = SampleGenerator('gaussian', image.size, opts['trans_f'], opts['scale_f'], valid=True)
    pos_generator = SampleGenerator('gaussian', image.size, 0.1, 1.2)
    neg_generator = SampleGenerator('uniform', image.size, 1.5, 1.2)

    # Init pos/neg features for update
    pos_feats_all = [pos_feats[:opts['n_pos_update']]]
    neg_feats_all = [neg_feats[:opts['n_neg_update']]]
    
    spf_total = time.time()-tic

    # Display
    savefig = savefig_dir != ''
    if display or savefig: 
        dpi = 80.0
        figsize = (image.size[0]/dpi, image.size[1]/dpi)

        fig = plt.figure(frameon=False, figsize=figsize, dpi=dpi)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        im = ax.imshow(image, aspect='equal')#, aspect='normal'

        if gt is not None:
            gt_rect = plt.Rectangle(tuple(gt[0,:2]),(gt[0,2]),(gt[0,3]), 
                    linewidth=3, edgecolor="#00ff00", zorder=1, fill=False)
            ax.add_patch(gt_rect)
       
        rect = plt.Rectangle(tuple((result_bb[0,:2])),(result_bb[0,2]),(result_bb[0,3]), 
                linewidth=3, edgecolor="#ff0000", zorder=1, fill=False)
        ax.add_patch(rect)

        if display:
            plt.pause(.01)
            plt.draw()
        if savefig:
            save_fig(image, result_bb[0,:], gt[0,:], 0, type = 0)
            #fig.savefig(os.path.join(savefig_dir,'0000.jpg'),dpi=dpi)#
    
    # Main loop
    for i in range(1,len(img_list)):

        tic = time.time()
        # Load image
        image = Image.open(img_list[i]).convert('RGB')

        # Estimate target bbox
        samples = gen_samples(sample_generator, target_bbox, opts['n_samples'])
        sample_scores = forward_samples(model, image, samples, out_layer='fc6')
        top_scores, top_idx = sample_scores[:,1].topk(5)
        top_idx = top_idx.cpu().numpy()
        target_score = top_scores.mean()
        target_bbox = samples[top_idx].mean(axis=0)

        success = target_score > opts['success_thr']
        
        # Expand search area at failure
        if success:
            sample_generator.set_trans_f(opts['trans_f'])
            success_fra.append(i)
        else:
            sample_generator.set_trans_f(opts['trans_f_expand'])

        # Bbox regression
        if success:
            bbreg_samples = samples[top_idx]
            bbreg_feats = forward_samples(model, image, bbreg_samples)
            bbreg_samples = bbreg.predict(bbreg_feats, bbreg_samples)
            bbreg_bbox = bbreg_samples.mean(axis=0)
        else:
            bbreg_bbox = target_bbox
        
        # Copy previous result at failure
        if not success:
            target_bbox = result[i-1]
            bbreg_bbox = result_bb[i-1]
        
        # Save result
        result[i] = target_bbox
        result_bb[i] = bbreg_bbox

        # Data collect
        if success:
            # Draw pos/neg samples
            pos_examples = gen_samples(pos_generator, target_bbox, 
                                       opts['n_pos_update'],
                                       opts['overlap_pos_update'])
            neg_examples = gen_samples(neg_generator, target_bbox, 
                                       opts['n_neg_update'],
                                       opts['overlap_neg_update'])

            # Extract pos/neg features
            pos_feats = forward_samples(model, image, pos_examples)
            neg_feats = forward_samples(model, image, neg_examples)
            pos_feats_all.append(pos_feats)
            neg_feats_all.append(neg_feats)
            if len(pos_feats_all) > opts['n_frames_long']:
                del pos_feats_all[0]
            if len(neg_feats_all) > opts['n_frames_short']:
                del neg_feats_all[0]

        # Short term update
        if not success:
            nframes = min(opts['n_frames_short'],len(pos_feats_all))
            pos_data = torch.stack(pos_feats_all[-nframes:],0).view(-1,feat_dim)
            neg_data = torch.stack(neg_feats_all,0).view(-1,feat_dim)
            train(model, criterion, update_optimizer, pos_data, neg_data, opts['maxiter_update'])
        
        # Long term update
        elif i % opts['long_interval'] == 0:
            pos_data = torch.stack(pos_feats_all,0).view(-1,feat_dim)
            neg_data = torch.stack(neg_feats_all,0).view(-1,feat_dim)
            train(model, criterion, update_optimizer, pos_data, neg_data, opts['maxiter_update'])
        
        spf = time.time()-tic
        spf_total += spf

        # Display
        if display or savefig:
            im.set_data(image)

            if gt is not None:
                gt_rect.set_xy(gt[i,:2])
                gt_rect.set_width(gt[i,2])
                gt_rect.set_height(gt[i,3])

            rect.set_xy(result_bb[i,:2])
            rect.set_width(result_bb[i,2])
            rect.set_height(result_bb[i,3])
            
            if display:
                plt.pause(.01)
                plt.draw()
            if savefig:
                save_fig(image, result_bb[i,:], gt[i,:], i, type = 0)
#                 draw = ImageDraw.Draw(image)
#                 draw.line([result_bb[i,0], result_bb[i,1], result_bb[i,0], result_bb[i,1] + result_bb[i,3], result_bb[i,0] + result_bb[i,2], result_bb[i,1] + result_bb[i,3], result_bb[i,0] + result_bb[i,2], result_bb[i,1], result_bb[i,0], result_bb[i,1]], fill = (0,255,0), width = 2)
#                 draw.line([gt[i,0], gt[i,1], gt[i,0], gt[i,1] + gt[i,3], gt[i,0] + gt[i,2], gt[i,1] + gt[i,3], gt[i,0] + gt[i,2], gt[i,1], gt[i,0], gt[i,1]], fill = (255,0,0), width = 2)
#                 del draw
#                 image.save(os.path.join(savefig_dir,'%04d.jpg'%(i)), "JPEG")
#                 #fig.savefig(os.path.join(savefig_dir,'%04d.jpg'%(i)))#,dpi=dpi

        if gt is None:
            print "Frame %d/%d, Score %.3f, Time %.3f" % \
                (i, len(img_list), target_score, spf)
        else:
            print "Frame %d/%d, Overlap %.3f, Score %.3f, Time %.3f" % \
                (i, len(img_list), overlap_ratio(gt[i],result_bb[i])[0], target_score, spf)

    fps = len(img_list) / spf_total
    print success_fra
    return result, result_bb, fps

In [8]:
def gen_config_ju(seq, savefig = False, json = '', display = False):
    if seq != '':
        # generate config from a sequence name
        
        seq_home = '../dataset/OTB'
        save_home = '../result_fig'
        result_home = '../result'
        
        seq_name = seq
        img_dir = os.path.join(seq_home, seq_name, 'img')
        gt_path = os.path.join(seq_home, seq_name, 'groundtruth_rect.txt')

        img_list = os.listdir(img_dir)
        img_list.sort()
        img_list = [os.path.join(img_dir,x) for x in img_list]

        gt = np.loadtxt(gt_path,delimiter=',')
        init_bbox = gt[0]
        
        savefig_dir = os.path.join(save_home,seq_name)
        result_dir = os.path.join(result_home,seq_name)
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
        result_path = os.path.join(result_dir,'result.json')

    elif json != '':
        # load config from a json file

        param = json.load(open(json,'r'))
        seq_name = param['seq_name']
        img_list = param['img_list']
        init_bbox = param['init_bbox']
        savefig_dir = param['savefig_dir']
        result_path = param['result_path']
        gt = None
        
    if savefig:
        if not os.path.exists(savefig_dir):
            os.makedirs(savefig_dir)
    else:
        savefig_dir = ''

    return img_list, init_bbox, gt, savefig_dir, display, result_path

In [9]:
# Generate sequence config
img_list, init_bbox, gt, savefig_dir, display, result_path = gen_config_ju('DragonBaby', savefig = True)

In [10]:
# Run tracker
result, result_bb, fps = run_mdnet(img_list, init_bbox, gt=gt, savefig_dir=savefig_dir, display=display)
    
# Save result
res = {}
res['res'] = result_bb.round().tolist()
res['type'] = 'rect'
res['fps'] = fps
json.dump(res, open(result_path, 'w'), indent=2)

Frame 1/113, Overlap 0.906, Score 5.378, Time 0.386
Frame 2/113, Overlap 0.773, Score 5.437, Time 0.392
Frame 3/113, Overlap 0.822, Score 4.981, Time 0.390
Frame 4/113, Overlap 0.889, Score 3.609, Time 0.392
Frame 5/113, Overlap 0.816, Score 3.343, Time 0.394
Frame 6/113, Overlap 0.849, Score 3.691, Time 0.395
Frame 7/113, Overlap 0.765, Score 2.839, Time 0.396
Frame 8/113, Overlap 0.778, Score 2.356, Time 0.392
Frame 9/113, Overlap 0.833, Score 3.044, Time 0.392
Frame 10/113, Overlap 0.765, Score 3.285, Time 0.620
Frame 11/113, Overlap 0.834, Score 5.891, Time 0.394
Frame 12/113, Overlap 0.796, Score 6.062, Time 0.401
Frame 13/113, Overlap 0.861, Score 6.090, Time 0.403
Frame 14/113, Overlap 0.794, Score 5.621, Time 0.408
Frame 15/113, Overlap 0.799, Score 5.287, Time 0.411
Frame 16/113, Overlap 0.813, Score 4.530, Time 0.414
Frame 17/113, Overlap 0.847, Score 6.013, Time 0.413
Frame 18/113, Overlap 0.800, Score 6.266, Time 0.413
Frame 19/113, Overlap 0.695, Score 5.741, Time 0.414
Fr

In [11]:
print fps

2.0515891374
